In [1]:
import os
from pathlib import Path
import re
from datetime import datetime

def rename_ndvi_files(base_folders: list, dry_run: bool = False) -> None:
    """
    Rename NDVI files in multiple folders from 'img_infer_*_YYYY-MM-DD.tif' to 'ndvi8days_YYYY-MM-DD.tif'
    
    Args:
        base_folders: List of base folder paths
        dry_run: If True, only print planned actions without executing them
    """
    # Compile regex pattern to extract date
    date_pattern = re.compile(r'(\d{4}-\d{2}-\d{2})\.tif$')
    
    for base_folder in base_folders:
        ndvi_folder = os.path.join(base_folder, 'ndvi')
        
        if not os.path.exists(ndvi_folder):
            print(f"\nSkipping: NDVI folder not found in {base_folder}")
            continue
            
        print(f"\nProcessing folder: {ndvi_folder}")
        
        # Get all tif files in the ndvi folder
        tif_files = list(Path(ndvi_folder).glob('img_infer_*.tif'))
        
        if not tif_files:
            print("No matching files found.")
            continue
            
        for file_path in tif_files:
            # Extract date from filename
            date_match = date_pattern.search(file_path.name)
            if date_match:
                date_str = date_match.group(1)
                new_name = f'ndvi8days_{date_str}.tif'
                new_path = file_path.parent / new_name
                
                print(f"Found: {file_path.name}")
                print(f"Will rename to: {new_name}")
                
                if not dry_run:
                    try:
                        file_path.rename(new_path)
                        print(f"Renamed successfully")
                    except Exception as e:
                        print(f"Error renaming file: {str(e)}")
                
            else:
                print(f"Skipping {file_path.name}: Could not extract date")

# Example usage
if __name__ == "__main__":
    # List of base folders to process
    folders = [
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/GiaiXuan-TanKy-NgheAn",
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/HiepHoa-KinhMon-HaiDuong",
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/ThanhBinh-TrangBom-DongNai",
        "/mnt/storage/data/EOV_LST/Train_LST/Data_train_1/ThuyVan-ThaiThuy-ThaiBinh"
        # Add more folders as needed
    ]
    
    # First do a dry run to see what would happen
    print("Performing dry run...")
    rename_ndvi_files(folders, dry_run=True)
    
    # Ask for confirmation before proceeding
    response = input("\nDo you want to proceed with the actual renaming? (yes/no): ")
    if response.lower() == 'yes':
        print("\nRenaming files...")
        rename_ndvi_files(folders, dry_run=False)
    else:
        print("\nOperation cancelled.")

Performing dry run...

Processing folder: /mnt/storage/data/EOV_LST/Train_LST/Data_train_1/GiaiXuan-TanKy-NgheAn/ndvi
Found: img_infer_giaixuan_2022-10-01.tif
Will rename to: ndvi8days_2022-10-01.tif
Found: img_infer_giaixuan_2022-10-17.tif
Will rename to: ndvi8days_2022-10-17.tif
Found: img_infer_giaixuan_2022-11-02.tif
Will rename to: ndvi8days_2022-11-02.tif
Found: img_infer_giaixuan_2022-11-18.tif
Will rename to: ndvi8days_2022-11-18.tif
Found: img_infer_giaixuan_2022-12-04.tif
Will rename to: ndvi8days_2022-12-04.tif
Found: img_infer_giaixuan_2022-12-20.tif
Will rename to: ndvi8days_2022-12-20.tif
Found: img_infer_giaixuan_2023-01-05.tif
Will rename to: ndvi8days_2023-01-05.tif
Found: img_infer_giaixuan_2023-01-21.tif
Will rename to: ndvi8days_2023-01-21.tif
Found: img_infer_giaixuan_2023-02-06.tif
Will rename to: ndvi8days_2023-02-06.tif
Found: img_infer_giaixuan_2023-02-22.tif
Will rename to: ndvi8days_2023-02-22.tif
Found: img_infer_giaixuan_2023-03-10.tif
Will rename to: ndvi8

In [1]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


In [2]:
def read_raster(filepath):
    """Đọc ảnh raster từ tệp .tif"""
    with rasterio.open(filepath) as src:
        data = src.read(1)  # Chỉ lấy kênh đầu tiên
        profile = src.profile
        transform = src.transform
    return data, profile, transform

def resample_raster(data, profile, target_shape):
    """Nội suy ảnh raster về kích thước mới (upscale)"""
    upscale_factor = target_shape[0] / data.shape[0]
    new_transform = profile["transform"] * profile["transform"].scale(upscale_factor)
    
    new_data = np.empty(target_shape, dtype=np.float32)
    
    reproject(
        source=data,
        destination=new_data,
        src_transform=profile["transform"],
        dst_transform=new_transform,
        src_crs=profile["crs"],
        dst_crs=profile["crs"],
        resampling=Resampling.bilinear,
    )
    return new_data


In [3]:
def sifm_downscaling(lr_sm_path, hr_ndvi_path, output_path):
    """
    Downscale độ ẩm đất từ ảnh độ phân giải thấp (LR) lên độ phân giải cao (HR) 
    bằng cách sử dụng ảnh NDVI có độ phân giải cao.
    """
    # Bước 1: Đọc ảnh LR (Soil Moisture) và HR (NDVI)
    lr_sm, lr_profile, lr_transform = read_raster(lr_sm_path)
    hr_ndvi, hr_profile, hr_transform = read_raster(hr_ndvi_path)
    
    # Bước 2: Nội suy ảnh LR lên độ phân giải cao của ảnh NDVI
    lr_sm_upsampled = resample_raster(lr_sm, lr_profile, hr_ndvi.shape)
    
    # Bước 3: Xây dựng mô hình hồi quy giữa LR_SM và HR_NDVI
    X = hr_ndvi.flatten().reshape(-1, 1)  # NDVI HR (biến độc lập)
    y = lr_sm_upsampled.flatten()  # Soil Moisture LR nội suy (biến phụ thuộc)
    
    regressor = LinearRegression()
    regressor.fit(X, y)  # Huấn luyện mô hình
    
    # Bước 4: Áp dụng mô hình để tạo HR Soil Moisture
    hr_sm_pred = regressor.predict(X).reshape(hr_ndvi.shape)
    
    # Bước 5: Lưu ảnh độ phân giải cao xuống
    hr_profile.update(dtype=rasterio.float32, count=1)
    
    with rasterio.open(output_path, 'w', **hr_profile) as dst:
        dst.write(hr_sm_pred.astype(np.float32), 1)

    print(f"Downscaling hoàn tất! Ảnh đã lưu tại {output_path}")
    
    return hr_sm_pred
